In [1]:
import BinanceTradeClient
import asyncio

import nest_asyncio

nest_asyncio.apply()

In [3]:
obj = BinanceTradeClient.FuturesOrder()

In [4]:
asyncio.run(obj.set_leverage(symbol='XRPUSDT', leverage=15))

{'symbol': 'XRPUSDT', 'leverage': 15, 'maxNotionalValue': '2000000'}

In [5]:
asyncio.run(obj.set_margin_type(symbol='XRPUSDT', margin_type='ISOLATED'))

'마진 타입이 이미 설정된 상태입니다 - ISOLATED'

In [6]:
asyncio.run(obj.get_account_balance())

{}

In [7]:
asyncio.run(obj.get_min_trade_quantity(symbol='XRPUSDT'))

2.0

In [10]:
asyncio.run(obj.submit_transfer(asset='USDT', amount=19, transfer_type=2))

{'tranId': 225135331171}